In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import sys; sys.path.insert(0, '..')

from utils.CategoricalStatistics import CategoricalStatistics
from utils.ContinuousStatistics import ContinuousStatistics

# Setup Mongo Database

In [ ]:
from pymongo import MongoClient

# Create a MongoClient to the running mongod instance
client = MongoClient('localhost', 27017)

# Get a reference to a particular database
db = client['pipeline']
feature_collection = db['feature']
label_collection = db['label']
feature_stats_collection = db['feature_stats']
label_stats_collection = db['label_stats']
data_collection = db['data']
datapoint_collection = db['datapoint']
model_collection = db['model']
scores_collection = db['scores']
train_process_collection = db['train_process']

# Initiale Datevorbereitung

In [ ]:
red = pd.read_csv("../data/base/winequality-red.csv", delimiter=";")
white = pd.read_csv("../data/base/winequality-white.csv", delimiter=";")


In [ ]:
label = "quality"

In [ ]:
white[label]  = white.quality.astype('category')

In [ ]:
white.head()

In [ ]:
stats = ContinuousStatistics(white["alcohol"]); stats.__dict__

In [ ]:
data_collection.insert_one({"type": "full", "location": {"src": "./data/base/winequality-white.csv"}})

In [ ]:
out = data_collection.find_one({"type": "full"})

In [ ]:
data_id_full = out['_id']

In [ ]:
white.columns

## Define Feature Descriptions

In [ ]:
descriptions = {
    "fixed acidity": "It refers to the concentration of non-volatile acids present in the wine, primarily tartaric acid. Fixed acidity contributes to the overall taste and structure of the wine, providing a sense of crispness or freshness.",
    "volatile acidity": "This property represents the amount of volatile acids, such as acetic acid, in the wine. Higher levels of volatile acidity can give the wine a vinegary or unpleasantly sharp taste.",
    "citric acid": "It is a type of fixed acid found in wines, derived from citrus fruits. Citric acid contributes to the wine's overall acidity and can enhance the freshness and fruitiness of the flavor profile.",
    "residual sugar": "It indicates the amount of sugar that remains unfermented in the wine. Wines with higher residual sugar levels tend to be sweeter, while those with lower levels are typically drier.",
    "chlorides": "This property refers to the concentration of chloride ions in the wine. Chlorides can contribute to the taste and mouthfeel of the wine, and their presence in excess can result in a salty or briny flavor.",
    "free sulfur dioxide": "It represents the amount of sulfur dioxide (SO2) that is not bound to other wine compounds. Free sulfur dioxide acts as an antioxidant and antimicrobial agent, helping to preserve the wine and prevent spoilage.",
    "total sulfur dioxide": "This property refers to the combined amount of free and bound sulfur dioxide present in the wine. It is measured in parts per million (ppm). Total sulfur dioxide levels are regulated and controlled, as excessive amounts can lead to off-flavors and health concerns.",
    "density": "It is a measure of the wine's mass per unit volume. Density is influenced by various factors, including alcohol content, residual sugar, and dissolved solids. It provides information about the wine's body and can be used to assess its overall quality.",
    "pH": "pH measures the acidity or alkalinity of the wine on a scale from 0 to 14. A pH below 7 indicates acidity, while a pH above 7 indicates alkalinity. Wine pH affects its taste, stability, and ability to age.",
    "sulphates": "This property represents the concentration of sulfur compounds, primarily sulfites, in the wine. Sulphates are often added as preservatives to prevent oxidation and microbial growth. Some individuals may be sensitive to high levels of sulfites.",
    "alcohol": "It refers to the percentage of alcohol by volume (ABV) in the wine. Alcohol content greatly influences the wine's body, mouthfeel, and perceived sweetness or heat. It is typically listed on the wine label and is an important factor in determining wine style.",
}

In [ ]:
for column in white.columns:
    values = white[column]
    tmp_dtype = values.dtype
    if tmp_dtype != "category":
        feature_collection.insert_one({
            "name": column, "description": descriptions[column], "dtype": str(tmp_dtype)})

In [ ]:
label_collection.insert_one({"name": "quality", "description": "each sample was evaluated by a minimum of three sensory assessors (using blind tastes), which graded the wine in a scale that ranges from 0 (very bad) to 10 (excellent)", "dtype": "categorical"})

In [ ]:
y = white[label]
X = white.drop(columns=label, inplace=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
X_test.head()

In [ ]:
features = white.columns
fig, axes = plt.subplots(4, 3, figsize=(15, 15))
axes = axes.ravel()

# Get unique categories
categories = white[features[-1]].unique()

for i in range(len(features) - 1):  # -1 because the last one is categorical
    # For each unique category, plot a histogram of this feature
    for category in categories:
        subset = white[white[features[-1]] == category]
        axes[i].hist(subset[features[i]], bins=30, label=str(category), alpha=0.7)
        
    axes[i].set_title(features[i])
    axes[i].legend()

# Plotting the categorical feature using seaborn countplot
sns.countplot(ax=axes[-1], x=white[features[-1]])
axes[-1].set_title(features[-1])

plt.tight_layout()
plt.show()

In [ ]:
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

train.head()

In [ ]:
data_collection.insert_one({"type": "train"})
data_collection.insert_one({"type": "test"})

In [ ]:
datasets = [X, X_train, X_test]
for i, data_type in enumerate(["full", "train", "test"]):
    data = datasets[i]
    for feature in data.columns:
        feature_id = feature_collection.find_one({"name": feature})['_id']
        data_id = data_collection.find_one({"type": data_type})['_id']
        statistics = ContinuousStatistics(data[feature])
        feature_stats_collection.insert_one({"feature_id": feature_id, "data_id": data_id, "statistics": statistics.__dict__})

In [ ]:
datasets = [y, y_train, y_test]

for i, data_type in enumerate(["full", "train", "test"]):
    data = datasets[i]
    label_id = label_collection.find_one({"name": label})['_id']
    data_id = data_collection.find_one({"type": data_type})['_id']
    statistics = CategoricalStatistics(data)
    label_stats_collection.insert_one({"label_id": label_id, "data_id": data_id, "statistics": statistics.__dict__})

# Write datapoints to database

In [ ]:
train_np = train.to_numpy()
test_np = test.to_numpy()

In [ ]:
train_data_id = data_collection.find_one({"type": "train"})
for index, elem in enumerate(train_np):
    datapoint_collection.insert_one(
        {
            "data_id": train_data_id,
            "index": index,
            'values': elem.tolist(),
        }
    )

In [ ]:
test_data_id = data_collection.find_one({"type": "test"})
for index, elem in enumerate(test_np):
    datapoint_collection.insert_one(
        {
            "data_id": test_data_id,
            "index": index,
            'values': elem.tolist(),
        }
    )

# Save processed data as csv

In [ ]:
import os
os.makedirs("./data/base", exist_ok=True)

In [ ]:
train.to_csv("./data/base/train-white.csv", index=False)
test.to_csv("./data/base/test-white.csv", index=False)